## Seldon V2 Kubernetes Examples


In [1]:
import os
os.environ["NAMESPACE"] = "seldon-mesh"

In [2]:
MESH_IP=!kubectl get svc seldon-mesh -n ${NAMESPACE} -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

'172.19.255.1'

### Model

In [3]:
!cat ./models/sklearn-iris-gs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn
  memory: 100Ki


In [4]:
!kubectl create -f ./models/sklearn-iris-gs.yaml -n ${NAMESPACE}

model.mlops.seldon.io/iris created


In [5]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/iris condition met


In [6]:
!kubectl get model iris -n ${NAMESPACE} -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-11-16T18:11:41Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-11-16T18:11:41Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [7]:
!seldon model infer iris --inference-host ${MESH_IP}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "de1b3d19-3fcb-4865-b59b-615a5b5f1e69",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				2
			]
		}
	]
}


In [8]:
!seldon model infer iris --inference-mode grpc --inference-host ${MESH_IP}:80 \
   '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' | jq -M .

{
  "modelName": "iris_1",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [9]:
!kubectl get server mlserver -n ${NAMESPACE} -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-11-16T18:06:01Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-11-16T18:06:01Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 1
}


In [10]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml -n ${NAMESPACE}

model.mlops.seldon.io "iris" deleted


### Experiment

In [11]:
!cat ./models/sklearn1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [12]:
!cat ./models/sklearn2.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris2
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [13]:
!kubectl create -f ./models/sklearn1.yaml -n ${NAMESPACE}
!kubectl create -f ./models/sklearn2.yaml -n ${NAMESPACE}

model.mlops.seldon.io/iris created
model.mlops.seldon.io/iris2 created


In [14]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/iris condition met
model.mlops.seldon.io/iris2 condition met


In [15]:
!cat ./experiments/ab-default-model.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
spec:
  default: iris
  candidates:
  - name: iris
    weight: 50
  - name: iris2
    weight: 50


In [16]:
!kubectl create -f ./experiments/ab-default-model.yaml -n ${NAMESPACE}

experiment.mlops.seldon.io/experiment-sample created


In [17]:
!kubectl wait --for condition=ready --timeout=300s experiment --all -n ${NAMESPACE}

experiment.mlops.seldon.io/experiment-sample condition met


In [18]:
!seldon model infer --inference-host ${MESH_IP}:80 -i 50 iris \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

Success: map[:iris2_1::27 :iris_1::23]


In [19]:
!kubectl delete -f ./experiments/ab-default-model.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/sklearn1.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/sklearn2.yaml -n ${NAMESPACE}

experiment.mlops.seldon.io "experiment-sample" deleted
model.mlops.seldon.io "iris" deleted
model.mlops.seldon.io "iris2" deleted


### Pipeline - model chain

In [20]:
!cat ./models/tfsimple1.yaml 
!cat ./models/tfsimple2.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki


In [28]:
!kubectl create -f ./models/tfsimple1.yaml -n ${NAMESPACE}
!kubectl create -f ./models/tfsimple2.yaml -n ${NAMESPACE}

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created


In [29]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/tfsimple1 condition met
model.mlops.seldon.io/tfsimple2 condition met


In [30]:
!cat ./pipelines/tfsimples.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: tfsimples
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
      inputs:
      - tfsimple1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple1.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple2


In [31]:
!kubectl create -f ./pipelines/tfsimples.yaml -n ${NAMESPACE}

pipeline.mlops.seldon.io/tfsimples created


In [32]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n ${NAMESPACE}

pipeline.mlops.seldon.io/tfsimples condition met


In [33]:
!seldon pipeline infer tfsimples --inference-mode grpc --inference-host ${MESH_IP}:80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ]
}


In [34]:
!kubectl delete -f ./pipelines/tfsimples.yaml -n ${NAMESPACE}

pipeline.mlops.seldon.io "tfsimples" deleted


In [35]:
!kubectl delete -f ./models/tfsimple1.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/tfsimple2.yaml -n ${NAMESPACE}

model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted


### Pipeline - model join

In [36]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml
!cat ./models/tfsimple3.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple3
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki


In [37]:
!kubectl create -f ./models/tfsimple1.yaml -n ${NAMESPACE}
!kubectl create -f ./models/tfsimple2.yaml -n ${NAMESPACE}
!kubectl create -f ./models/tfsimple3.yaml -n ${NAMESPACE}

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created
model.mlops.seldon.io/tfsimple3 created


In [38]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/tfsimple1 condition met
model.mlops.seldon.io/tfsimple2 condition met
model.mlops.seldon.io/tfsimple3 condition met


In [39]:
!cat ./pipelines/tfsimples-join.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: join
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
    - name: tfsimple3      
      inputs:
      - tfsimple1.outputs.OUTPUT0
      - tfsimple2.outputs.OUTPUT1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple2.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple3


In [40]:
!kubectl create -f ./pipelines/tfsimples-join.yaml -n ${NAMESPACE}

pipeline.mlops.seldon.io/join created


In [41]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n ${NAMESPACE}

pipeline.mlops.seldon.io/join condition met


In [42]:
!seldon pipeline infer join --inference-mode grpc --inference-host ${MESH_IP}:80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ]
}


In [43]:
!kubectl delete -f ./pipelines/tfsimples-join.yaml -n ${NAMESPACE}

pipeline.mlops.seldon.io "join" deleted


In [44]:
!kubectl delete -f ./models/tfsimple1.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/tfsimple2.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/tfsimple3.yaml -n ${NAMESPACE}

model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted
model.mlops.seldon.io "tfsimple3" deleted


## Explainer

In [45]:
!cat ./models/income.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/classifier"
  requirements:
  - sklearn


In [46]:
!kubectl create -f ./models/income.yaml -n ${NAMESPACE}

model.mlops.seldon.io/income created


In [47]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/income condition met


In [48]:
!kubectl get model income -n ${NAMESPACE} -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-11-16T18:16:34Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-11-16T18:16:34Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [49]:
!seldon model infer income --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income_1",
	"model_version": "1",
	"id": "11e439cf-c967-44fd-aa13-baefd9c4d407",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				0
			]
		}
	]
}


In [50]:
!cat ./models/income-explainer.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-explainer
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/explainer"
  explainer:
    type: anchor_tabular
    modelRef: income


In [51]:
!kubectl create -f ./models/income-explainer.yaml -n ${NAMESPACE}

model.mlops.seldon.io/income-explainer created


In [52]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/income condition met
model.mlops.seldon.io/income-explainer condition met


In [53]:
!kubectl get model income-explainer -n ${NAMESPACE} -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-11-16T18:16:51Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-11-16T18:16:51Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [54]:
!seldon model infer income-explainer --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income-explainer_1",
	"model_version": "1",
	"id": "7aa0f8ec-a68a-4141-a757-56ebd39f2845",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "explanation",
			"shape": [
				1
			],
			"datatype": "BYTES",
			"parameters": {
				"content_type": "str",
				"headers": null
			},
			"data": [
				"{\"meta\": {\"name\": \"AnchorTabular\", \"type\": [\"blackbox\"], \"explanations\": [\"local\"], \"params\": {\"seed\": 1, \"disc_perc\": [25, 50, 75], \"threshold\": 0.95, \"delta\": 0.1, \"tau\": 0.15, \"batch_size\": 100, \"coverage_samples\": 10000, \"beam_size\": 1, \"stop_on_first\": false, \"max_anchor_size\": null, \"min_samples_start\": 100, \"n_covered_ex\": 10, \"binary_cache_size\": 10000, \"cache_margin\": 1000, \"verbose\": false, \"verbose_every\": 1, \"kwargs\": {}}, \"version\": \"0.8.0\"}, \"data\": {\"anchor\": [\"Marital Status = Never-Married\", \"Occupation = Admin\", \"Capital Gain <= 0.00\", \"Capital Loss <= 0.

In [55]:
!kubectl delete -f ./models/income.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/income-explainer.yaml -n ${NAMESPACE}

model.mlops.seldon.io "income" deleted
model.mlops.seldon.io "income-explainer" deleted


## Custom Server

In [16]:
!cat ./servers/custom-mlserver.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Server
metadata:
  name: mlserver-custom
spec:
  serverConfig: mlserver
  podSpec:
    containers:
    - image: cliveseldon/mlserver:1.2.0.dev1
      name: mlserver


In [17]:
!kubectl create -f ./servers/custom-mlserver.yaml -n ${NAMESPACE}

server.mlops.seldon.io/mlserver-custom created


In [18]:
!kubectl wait --for condition=ready --timeout=300s server --all -n ${NAMESPACE}

server.mlops.seldon.io/mlserver condition met
server.mlops.seldon.io/mlserver-custom condition met
server.mlops.seldon.io/triton condition met


In [19]:
!cat ./models/iris-custom-server.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  server: mlserver-custom


In [20]:
!kubectl create -f ./models/iris-custom-server.yaml -n ${NAMESPACE}

model.mlops.seldon.io/iris created


In [21]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/iris condition met


In [22]:
!seldon model infer iris --inference-host ${MESH_IP}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "ddfb14fa-dd0e-4960-9a89-4137570f5feb",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				2
			]
		}
	]
}


In [23]:
!kubectl delete -f ./models/iris-custom-server.yaml -n ${NAMESPACE}

model.mlops.seldon.io "iris" deleted


In [24]:
!kubectl delete -f ./servers/custom-mlserver.yaml -n ${NAMESPACE}

server.mlops.seldon.io "mlserver-custom" deleted
